In [1]:
# importamos las librerias necesarias para la realización de la práctica
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
# introducimos la url desde donde haremos scraping
req = requests.get('https://www.imdb.com/chart/top?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=4da9d9a5-d299-43f2-9c53-f0efa18182cd&pf_rd_r=WTF1PM0FNNA7Z2KZCGFG&pf_rd_s=right-4&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_ql_3')

# si el status code es 200, significa que la página se ha cargado de forma correcta
if req.status_code == 200:
    print('OK. Conexión establecida')
    content = req.content
    
html = BeautifulSoup(req.text, "html.parser")

# aquí iremos guardando los diferentes datos. Tenemos 3 tipos de páginas diferentes que deberemos recorrer
datos_pagina1 = {}
datos_pagina2 = {}
datos_pagina3 = {}

OK. Conexión establecida


In [3]:
# contador de películas (nos servirá para identificar los puestos con alguna anomalía - datos incorrectos)
i=0

paginas1 = html.find_all('td', {'class': 'titleColumn'})

# recorremos el primer tipo de página web, donde extraeremos el 'puesto', 'título', 'año' y 'url'
# de la pélicula de donde extraeremos los siguientes datos
for pagina1 in paginas1:

    titulo = pagina1.find('a').getText()

    anyo = pagina1.find('span', {'class': 'secondaryInfo'}).getText().split('(', 1)[1].split(')', 5)[0]

    puesto = pagina1.getText().split()[0].split('.')[0]

    enlace = pagina1.find('a').get('href')

    url = 'https://www.imdb.com'+enlace+'?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_'+puesto
     
    i = i +1

# guardamos los datos en cada pasada
    datos_pagina1[i] = [puesto, titulo, anyo]
    
    print (puesto, titulo, anyo, url)
    
    req2 = requests.get(url)      

    html2 = BeautifulSoup(req2.text, "html.parser")

    paginas2 = html2.find_all('div', {'class': 'title_bar_wrapper'})
    
# recorremos el segundo tipo de página web, donde extraeremos el 'rating', 'votos', 'edad', 'duración', 'género'
# y 'url' de los rating de la pélicula por población

    for pagina2 in paginas2:

        ranting = pagina2.find('span', {'itemprop': 'ratingValue'}).getText()

        votos = pagina2.find('span', {'class': 'small'}).getText()

        enlace_votos = pagina2.find('a').get('href')

        url_votos = 'https://www.imdb.com'+enlace_votos

        edad = pagina2.find('div', {'class': 'subtext'}).getText().split('|')[0].split()[0]
        
        duracion = pagina2.find('time').getText().strip()
        
        genero_tmp = pagina2.find('div', {'class': 'subtext'}).getText().split('|')[2].strip()

        genero = re.sub(r"\s+", " ", genero_tmp)
        
# guardamos los datos en cada pasada
        datos_pagina2[i] = [ranting, votos, edad, duracion, genero]
        
        
        
        req3 = requests.get(url_votos)      

        html3 = BeautifulSoup(req3.text, "html.parser")
        
        paginas3 = html3.find_all('div', {'class': 'title-ratings-sub-page'})

# recorremos el el tercer tipo de página web, donde extraeremos el '<18', '18-29', '30-44', '45+'
# según los rating de la pélicula por población

        for pagina3 in paginas3:

            menos_18 = pagina3.find_all('td', {'class': 'ratingTable'})[1].getText().strip().split()[0]
            
            entre_18_29 = pagina3.find_all('td', {'class': 'ratingTable'})[2].getText().strip().split()[0]
            
            entre_30_44 = pagina3.find_all('td', {'class': 'ratingTable'})[3].getText().strip().split()[0]
            
            mas_45 = pagina3.find_all('td', {'class': 'ratingTable'})[4].getText().strip().split()[0]
            
            
            
# guardamos los datos en cada pasada           
            datos_pagina3[i] = [menos_18, entre_18_29, entre_30_44, mas_45]
                

    print (ranting, votos, url_votos, edad, duracion, genero, menos_18, entre_18_29, entre_30_44, mas_45)



1 Cadena perpetua 1994 https://www.imdb.com/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_1
9.3 2,201,192 https://www.imdb.com/title/tt0111161/ratings 13 2h 22min Drama 9.4 9.3 9.3 9.1
2 El padrino 1972 https://www.imdb.com/title/tt0068646/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_2
9.2 1,516,621 https://www.imdb.com/title/tt0068646/ratings 18 2h 55min Crime, Drama 9.2 9.2 9.2 9.1
3 El padrino: Parte II 1974 https://www.imdb.com/title/tt0071562/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_3
9.0 1,062,493 https://www.imdb.com/title/tt0071562/ratings 18 3h 22min Crime, Drama 9.1 9.1 9.0 8.9
4 El caballero oscuro 2008 https

8.6 317,775 https://www.imdb.com/title/tt6751668/ratings 16 2h 12min Comedy, Drama, Thriller 9.1 8.8 8.4 8.2
26 La guerra de las galaxias 1977 https://www.imdb.com/title/tt0076759/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_26
8.6 1,176,511 https://www.imdb.com/title/tt0076759/ratings A 2h 1min Action, Adventure, Fantasy 8.6 8.5 8.6 8.8
27 Salvar al soldado Ryan 1998 https://www.imdb.com/title/tt0120815/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_27
8.6 1,167,656 https://www.imdb.com/title/tt0120815/ratings 13 2h 49min Drama, War 8.7 8.7 8.5 8.5
28 El viaje de Chihiro 2001 https://www.imdb.com/title/tt0245429/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemet

8.5 495,974 https://www.imdb.com/title/tt0034583/ratings T 1h 42min Drama, Romance, War 8.5 8.4 8.4 8.7
51 La ventana indiscreta 1954 https://www.imdb.com/title/tt0047396/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_51
8.4 418,432 https://www.imdb.com/title/tt0047396/ratings T 1h 52min Mystery, Thriller 8.8 8.5 8.4 8.5
52 Cinema Paradiso 1988 https://www.imdb.com/title/tt0095765/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_52
8.5 213,343 https://www.imdb.com/title/tt0095765/ratings A 2h 35min Drama 8.7 8.6 8.4 8.4
53 Alien, el octavo pasajero 1979 https://www.imdb.com/title/tt0078748/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_53
8.4 747

8.4 160,379 https://www.imdb.com/title/tt5311514/ratings A 1h 46min Animation, Drama, Fantasy 8.7 8.6 8.2 7.9
76 American Beauty 1999 https://www.imdb.com/title/tt0169547/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_76
8.3 1,025,311 https://www.imdb.com/title/tt0169547/ratings 18 2h 2min Drama 8.8 8.5 8.3 8.1
77 Coco 2017 https://www.imdb.com/title/tt2380307/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_77
8.4 324,719 https://www.imdb.com/title/tt2380307/ratings A 1h 45min Animation, Adventure, Family 8.6 8.5 8.2 8.1
78 Braveheart 1995 https://www.imdb.com/title/tt0112573/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_78
8.3 922,103 https://

8.3 747,284 https://www.imdb.com/title/tt0208092/ratings 18 1h 44min Comedy, Crime 8.3 8.3 8.3 8.0
101 El chico 1921 https://www.imdb.com/title/tt0012349/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_101
8.3 104,344 https://www.imdb.com/title/tt0012349/ratings T 1h 8min Comedy, Drama, Family 8.5 8.4 8.3 8.3
102 Ladrón de bicicletas 1948 https://www.imdb.com/title/tt0040522/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_102
8.3 135,397 https://www.imdb.com/title/tt0040522/ratings T 1h 29min Drama 8.3 8.3 8.3 8.3
103 Cantando bajo la lluvia 1952 https://www.imdb.com/title/tt0045152/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_103
8.3 206,519 h

...

8.2 277,638 https://www.imdb.com/title/tt0071315/ratings A 2h 10min Drama, Mystery, Thriller 8.4 8.2 8.1 8.2
151 La quimera del oro 1925 https://www.imdb.com/title/tt0015864/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_151
8.2 93,748 https://www.imdb.com/title/tt0015864/ratings T 1h 35min Adventure, Comedy, Drama 8.3 8.2 8.1 8.2
152 Mi padre y mi hijo 2005 https://www.imdb.com/title/tt0476735/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_152
8.3 72,616 https://www.imdb.com/title/tt0476735/ratings 1h 1h 48min 18 November 2005 (Turkey) 8.0 8.5 8.3 7.3
153 Crimen perfecto 1954 https://www.imdb.com/title/tt0046912/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter

8.1 692,076 https://www.imdb.com/title/tt1205489/ratings 12 1h 56min Drama 8.2 8.2 8.1 8.0
176 Stalker 1979 https://www.imdb.com/title/tt0079944/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_176
8.2 105,434 https://www.imdb.com/title/tt0079944/ratings 2h 2h 42min 5 November 1984 (Spain) 8.7 8.4 8.1 7.8
177 El show de Truman (Una vida en directo) 1998 https://www.imdb.com/title/tt0120382/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_177
8.1 878,680 https://www.imdb.com/title/tt0120382/ratings T 1h 43min Comedy, Drama, Sci-Fi 8.6 8.3 8.1 7.7
178 El cazador 1978 https://www.imdb.com/title/tt0077416/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_

8.1 604,810 https://www.imdb.com/title/tt2024544/ratings 16 2h 14min Biography, Drama, History 8.4 8.3 8.0 7.8
201 La leyenda del indomable 1967 https://www.imdb.com/title/tt0061512/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_201
8.1 154,405 https://www.imdb.com/title/tt0061512/ratings 7 2h 7min Crime, Drama 8.0 8.1 8.1 8.2
202 Mad Max: Furia en la carretera 2015 https://www.imdb.com/title/tt1392190/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_202
8.1 824,370 https://www.imdb.com/title/tt1392190/ratings 16 2h Action, Adventure, Sci-Fi 8.4 8.3 8.0 7.6
203 Cuenta conmigo 1986 https://www.imdb.com/title/tt0092005/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemet

8.1 93,718 https://www.imdb.com/title/tt4016934/ratings 16 2h 25min Drama, Romance, Thriller 8.5 8.3 7.9 7.8
226 Rebeca 1940 https://www.imdb.com/title/tt0032976/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_226
8.1 115,421 https://www.imdb.com/title/tt0032976/ratings 13 2h 10min Drama, Mystery, Romance 8.1 8.1 8.0 8.2
227 Rocky 1976 https://www.imdb.com/title/tt0075148/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_227
8.1 493,003 https://www.imdb.com/title/tt0075148/ratings 18 2h Drama, Sport 8.2 8.1 8.0 8.0
228 Monstruos, S.A. 2001 https://www.imdb.com/title/tt0198781/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_228
8.0 774,808 https://ww

8.0 558,098 https://www.imdb.com/title/tt0107048/ratings T 1h 41min Comedy, Fantasy, Romance 8.0 7.9 8.1 8.1


In [4]:
datos_pagina1_df = pd.DataFrame.from_dict(datos_pagina1, orient = 'index', columns = ['Puesto','Titulo', 'Año'])

In [5]:
datos_pagina2_df = pd.DataFrame.from_dict(datos_pagina2, orient = 'index', columns = ['Rating', 'Votos', 'Edad','Duracion','Genero'])

In [6]:
datos_pagina3_df = pd.DataFrame.from_dict(datos_pagina3, orient = 'index', columns = ['<18','18-29', '30-44', '45+'])

In [7]:
# de esta forma quedaría creado el primer dataframe con el primer tipo de páginas web visitado
datos_pagina1_df

,Puesto,Titulo,Año
1,1,Cadena perpetua,1994
2,2,El padrino,1972
3,3,El padrino: Parte II,1974
4,4,El caballero oscuro,2008
5,5,12 hombres sin piedad,1957
...,...,...,...
246,246,Contratiempo,2016
247,247,Guardianes de la galaxia,2014
248,248,Neon Genesis Evangelion: The End of Evangelion,1997
249,249,Drishyam,2015


In [8]:
# de esta forma quedaría creado el segundo dataframe con el segundo tipo de páginas web visitado
datos_pagina2_df

,Rating,Votos,Edad,Duracion,Genero
1,9.3,"2,201,192",13,2h 22min,Drama
2,9.2,"1,516,621",18,2h 55min,"Crime, Drama"
3,9.0,"1,062,493",18,3h 22min,"Crime, Drama"
4,9.0,"2,182,674",12,2h 32min,"Action, Crime, Drama"
5,8.9,"642,713",A,1h 36min,Drama
...,...,...,...,...,...
246,8.1,"104,343",12,1h 46min,"Crime, Mystery, Thriller"
247,8.0,"1,000,430",12,2h 1min,"Action, Adventure, Comedy"
248,8.1,"33,330",13,1h 27min,"Animation, Action, Drama"
249,8.2,"60,285",2h,2h 43min,31 July 2015 (India)


In [9]:
# de esta forma quedaría creado el tercer dataframe con el tercer tipo de páginas web visitado
datos_pagina3_df

,<18,18-29,30-44,45+
1,9.4,9.3,9.3,9.1
2,9.2,9.2,9.2,9.1
3,9.1,9.1,9.0,8.9
4,9.3,9.2,9.0,8.4
5,9.3,9.1,8.9,8.7
...,...,...,...,...
246,8.2,8.3,8.0,7.8
247,8.3,8.1,8.0,7.9
248,8.5,8.4,8.0,6.7
249,8.2,8.5,7.9,6.4


In [10]:
# concatemos los tres dataframes
datos_df = pd.concat([datos_pagina1_df, datos_pagina2_df, datos_pagina3_df], axis=1)

In [11]:
# mostramos cómo quedaría el listado de los cinco primeros elementos del dataframe
datos_df.head()

,Puesto,Titulo,Año,Rating,Votos,Edad,Duracion,Genero,<18,18-29,30-44,45+
1,1,Cadena perpetua,1994,9.3,"2,201,192",13,2h 22min,Drama,9.4,9.3,9.3,9.1
2,2,El padrino,1972,9.2,"1,516,621",18,2h 55min,"Crime, Drama",9.2,9.2,9.2,9.1
3,3,El padrino: Parte II,1974,9.0,"1,062,493",18,3h 22min,"Crime, Drama",9.1,9.1,9.0,8.9
4,4,El caballero oscuro,2008,9.0,"2,182,674",12,2h 32min,"Action, Crime, Drama",9.3,9.2,9.0,8.4
5,5,12 hombres sin piedad,1957,8.9,"642,713",A,1h 36min,Drama,9.3,9.1,8.9,8.7


In [12]:
# Hemos observado que hay datos que no están completos o contiene errores, tienen alguna anomalía
# crearemos un filtro para ver qué elementos lo tienen. En este caso comprobamos cómo la variable 'Género' 
# plantea problemas, ya que no nos da el estilo de la película sino información del país y fecha de su estreno

# También hemos observado que el valor en la variable edad, la clasificación de la película, no lo tenemos
# Así que introduciremos un valor de 'NC' (No Clasificado) para dichos elementos
filtro = datos_df[datos_df['Genero'].str.contains("\d+", case=False)]
filtro

,Puesto,Titulo,Año,Rating,Votos,Edad,Duracion,Genero,<18,18-29,30-44,45+
33,33,Harakiri,1962,8.7,"31,150",2h,2h 13min,16 September 1962 (Japan),7.3,8.8,8.7,8.5
93,93,Dangal,2016,8.4,"136,974",2h,2h 41min,23 December 2016 (India),8.6,8.6,8.3,7.4
132,132,Masacre (ven y mira),1985,8.3,"48,480",2h,2h 22min,17 October 1985 (Hungary),8.4,8.5,8.2,8.1
135,135,Ran,1985,8.2,"104,312",2h,2h 42min,17 January 1986 (Spain),8.2,8.3,8.2,8.2
146,146,El tesoro de Sierra Madre,1948,8.2,"107,984",2h,2h 6min,24 January 1948 (USA),8.5,8.2,8.1,8.3
152,152,Mi padre y mi hijo,2005,8.3,"72,616",1h,1h 48min,18 November 2005 (Turkey),8.0,8.5,8.3,7.3
171,171,Fresas salvajes,1957,8.2,"88,704",1h,1h 31min,7 February 1963 (Spain),8.2,8.3,8.1,8.1
172,172,La ley del silencio,1954,8.1,"134,128",1h,1h 48min,22 June 1954 (Japan),8.1,8.2,8.1,8.2
176,176,Stalker,1979,8.2,"105,434",2h,2h 42min,5 November 1984 (Spain),8.7,8.4,8.1,7.8
182,182,Eskiya,1996,8.3,"58,945",2h,2h 8min,29 November 1996 (Turkey),6.9,8.4,8.4,6.9


In [13]:
# creamos un array con los puestos del ranking que la película ocupa
numero = filtro['Puesto'].values
numero

array(['33', '93', '132', '135', '146', '152', '171', '172', '176', '182',
       '184', '185', '204', '216', '219', '237', '239', '245', '249'],
      dtype=object)

In [14]:
# transformamos los string a tipo numérico 'int'. De esta forma podremos compararlo con el contador de la película
b= [int(x) for x in numero]
m = np.array(b)
m

array([ 33,  93, 132, 135, 146, 152, 171, 172, 176, 182, 184, 185, 204,
       216, 219, 237, 239, 245, 249])

In [15]:
# código similar al anterior, pero esta vez introducimos condicionales, para corregir los problemas que teníamos
# en el primer scraping que realizamos

i=0

paginas1 = html.find_all('td', {'class': 'titleColumn'})

for pagina1 in paginas1:

    titulo = pagina1.find('a').getText()

    anyo = pagina1.find('span', {'class': 'secondaryInfo'}).getText().split('(', 1)[1].split(')', 5)[0]

    puesto = pagina1.getText().split()[0].split('.')[0]

    enlace = pagina1.find('a').get('href')

    url = 'https://www.imdb.com'+enlace+'?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_'+puesto
     
    i = i +1
    
    datos_pagina1[i] = [puesto, titulo, anyo]
    
    print (puesto, titulo, anyo, url)
    
    req2 = requests.get(url)      

    html2 = BeautifulSoup(req2.text, "html.parser")

    paginas2 = html2.find_all('div', {'class': 'title_bar_wrapper'})

    for pagina2 in paginas2:

        ranting = pagina2.find('span', {'itemprop': 'ratingValue'}).getText()

        votos = pagina2.find('span', {'class': 'small'}).getText()

        enlace_votos = pagina2.find('a').get('href')

        url_votos = 'https://www.imdb.com'+enlace_votos

# si la 'edad' corresponde con alguno de los puestos que ocupa la película, introducimos el valor de 'NC'
# en caso contrario, extraemos el valor de forma normal
        
        if i in m:

            edad = 'NC'
            
        else:
            
            edad = pagina2.find('div', {'class': 'subtext'}).getText().split('|')[0].split()[0]
            
        
        duracion = pagina2.find('time').getText().strip()
        
# si el 'genero' corresponde con alguno de los puestos que ocupa la película, extraemos de esta forma los valores.
# En caso contrario, mostramos cómo hacerlo
        
        if i in m:
        
            genero_tmp = pagina2.find('div', {'class': 'subtext'}).getText().split('|')[1].strip()
            
        else:
            
            genero_tmp = pagina2.find('div', {'class': 'subtext'}).getText().split('|')[2].strip()
        
        
        genero = re.sub(r"\s+", " ", genero_tmp)
        
        
        datos_pagina2[i] = [ranting, votos, edad, duracion, genero]
        
        
        
        req3 = requests.get(url_votos)      

        html3 = BeautifulSoup(req3.text, "html.parser")
        
        paginas3 = html3.find_all('div', {'class': 'title-ratings-sub-page'})


        for pagina3 in paginas3:

            menos_18 = pagina3.find_all('td', {'class': 'ratingTable'})[1].getText().strip().split()[0]
            
            entre_18_29 = pagina3.find_all('td', {'class': 'ratingTable'})[2].getText().strip().split()[0]
            
            entre_30_44 = pagina3.find_all('td', {'class': 'ratingTable'})[3].getText().strip().split()[0]
            
            mas_45 = pagina3.find_all('td', {'class': 'ratingTable'})[4].getText().strip().split()[0]
            
            
            
            
            datos_pagina3[i] = [menos_18, entre_18_29, entre_30_44, mas_45]
                
        #print (menos_18)

    print (ranting, votos, url_votos, edad, duracion, genero, menos_18, entre_18_29, entre_30_44, mas_45)


1 Cadena perpetua 1994 https://www.imdb.com/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_1
9.3 2,201,192 https://www.imdb.com/title/tt0111161/ratings 13 2h 22min Drama 9.4 9.3 9.3 9.1
2 El padrino 1972 https://www.imdb.com/title/tt0068646/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_2
9.2 1,516,621 https://www.imdb.com/title/tt0068646/ratings 18 2h 55min Crime, Drama 9.2 9.2 9.2 9.1
3 El padrino: Parte II 1974 https://www.imdb.com/title/tt0071562/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_3
9.0 1,062,493 https://www.imdb.com/title/tt0071562/ratings 18 3h 22min Crime, Drama 9.1 9.1 9.0 8.9
4 El caballero oscuro 2008 https

8.6 317,775 https://www.imdb.com/title/tt6751668/ratings 16 2h 12min Comedy, Drama, Thriller 9.1 8.8 8.4 8.2
26 La guerra de las galaxias 1977 https://www.imdb.com/title/tt0076759/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_26
8.6 1,176,511 https://www.imdb.com/title/tt0076759/ratings A 2h 1min Action, Adventure, Fantasy 8.6 8.5 8.6 8.8
27 Salvar al soldado Ryan 1998 https://www.imdb.com/title/tt0120815/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_27
8.6 1,167,656 https://www.imdb.com/title/tt0120815/ratings 13 2h 49min Drama, War 8.7 8.7 8.5 8.5
28 El viaje de Chihiro 2001 https://www.imdb.com/title/tt0245429/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemet

8.5 495,974 https://www.imdb.com/title/tt0034583/ratings T 1h 42min Drama, Romance, War 8.5 8.4 8.4 8.7
51 La ventana indiscreta 1954 https://www.imdb.com/title/tt0047396/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_51
8.4 418,432 https://www.imdb.com/title/tt0047396/ratings T 1h 52min Mystery, Thriller 8.8 8.5 8.4 8.5
52 Cinema Paradiso 1988 https://www.imdb.com/title/tt0095765/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_52
8.5 213,343 https://www.imdb.com/title/tt0095765/ratings A 2h 35min Drama 8.7 8.6 8.4 8.4
53 Alien, el octavo pasajero 1979 https://www.imdb.com/title/tt0078748/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_53
8.4 747

8.4 160,379 https://www.imdb.com/title/tt5311514/ratings A 1h 46min Animation, Drama, Fantasy 8.7 8.6 8.2 7.9
76 American Beauty 1999 https://www.imdb.com/title/tt0169547/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_76
8.3 1,025,311 https://www.imdb.com/title/tt0169547/ratings 18 2h 2min Drama 8.8 8.5 8.3 8.1
77 Coco 2017 https://www.imdb.com/title/tt2380307/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_77
8.4 324,719 https://www.imdb.com/title/tt2380307/ratings A 1h 45min Animation, Adventure, Family 8.6 8.5 8.2 8.1
78 Braveheart 1995 https://www.imdb.com/title/tt0112573/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_78
8.3 922,103 https://

8.3 747,284 https://www.imdb.com/title/tt0208092/ratings 18 1h 44min Comedy, Crime 8.3 8.3 8.3 8.0
101 El chico 1921 https://www.imdb.com/title/tt0012349/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_101
8.3 104,344 https://www.imdb.com/title/tt0012349/ratings T 1h 8min Comedy, Drama, Family 8.5 8.4 8.3 8.3
102 Ladrón de bicicletas 1948 https://www.imdb.com/title/tt0040522/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_102
8.3 135,397 https://www.imdb.com/title/tt0040522/ratings T 1h 29min Drama 8.3 8.3 8.3 8.3
103 Cantando bajo la lluvia 1952 https://www.imdb.com/title/tt0045152/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_103
8.3 206,519 h

...

8.2 277,638 https://www.imdb.com/title/tt0071315/ratings A 2h 10min Drama, Mystery, Thriller 8.4 8.2 8.1 8.2
151 La quimera del oro 1925 https://www.imdb.com/title/tt0015864/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_151
8.2 93,748 https://www.imdb.com/title/tt0015864/ratings T 1h 35min Adventure, Comedy, Drama 8.3 8.2 8.1 8.2
152 Mi padre y mi hijo 2005 https://www.imdb.com/title/tt0476735/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_152
8.3 72,616 https://www.imdb.com/title/tt0476735/ratings NC 1h 48min Drama, Family 8.0 8.5 8.3 7.3
153 Crimen perfecto 1954 https://www.imdb.com/title/tt0046912/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm

8.1 692,076 https://www.imdb.com/title/tt1205489/ratings 12 1h 56min Drama 8.2 8.2 8.1 8.0
176 Stalker 1979 https://www.imdb.com/title/tt0079944/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_176
8.2 105,434 https://www.imdb.com/title/tt0079944/ratings NC 2h 42min Drama, Sci-Fi 8.7 8.4 8.1 7.8
177 El show de Truman (Una vida en directo) 1998 https://www.imdb.com/title/tt0120382/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_177
8.1 878,680 https://www.imdb.com/title/tt0120382/ratings T 1h 43min Comedy, Drama, Sci-Fi 8.6 8.3 8.1 7.7
178 El cazador 1978 https://www.imdb.com/title/tt0077416/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_178
8.1 29

8.1 604,810 https://www.imdb.com/title/tt2024544/ratings 16 2h 14min Biography, Drama, History 8.4 8.3 8.0 7.8
201 La leyenda del indomable 1967 https://www.imdb.com/title/tt0061512/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_201
8.1 154,405 https://www.imdb.com/title/tt0061512/ratings 7 2h 7min Crime, Drama 8.0 8.1 8.1 8.2
202 Mad Max: Furia en la carretera 2015 https://www.imdb.com/title/tt1392190/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_202
8.1 824,370 https://www.imdb.com/title/tt1392190/ratings 16 2h Action, Adventure, Sci-Fi 8.4 8.3 8.0 7.6
203 Cuenta conmigo 1986 https://www.imdb.com/title/tt0092005/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemet

8.1 93,718 https://www.imdb.com/title/tt4016934/ratings 16 2h 25min Drama, Romance, Thriller 8.5 8.3 7.9 7.8
226 Rebeca 1940 https://www.imdb.com/title/tt0032976/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_226
8.1 115,421 https://www.imdb.com/title/tt0032976/ratings 13 2h 10min Drama, Mystery, Romance 8.1 8.1 8.0 8.2
227 Rocky 1976 https://www.imdb.com/title/tt0075148/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_227
8.1 493,003 https://www.imdb.com/title/tt0075148/ratings 18 2h Drama, Sport 8.2 8.1 8.0 8.0
228 Monstruos, S.A. 2001 https://www.imdb.com/title/tt0198781/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&pf_rd_r=TGWT3G80CFT1NXRXKXB5&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=moviemeter&ref_=chtmvm_tt_228
8.0 774,808 https://ww

8.0 558,098 https://www.imdb.com/title/tt0107048/ratings T 1h 41min Comedy, Fantasy, Romance 8.0 7.9 8.1 8.1


In [16]:
datos_final1_df = pd.DataFrame.from_dict(datos_pagina1, orient = 'index', columns = ['Puesto','Titulo', 'Año'])

In [17]:
datos_final2_df = pd.DataFrame.from_dict(datos_pagina2, orient = 'index', columns = ['Rating', 'Votos', 'Edad','Duración','Género'])

In [18]:
datos_final3_df = pd.DataFrame.from_dict(datos_pagina3, orient = 'index', columns = ['<18','18-29', '30-44', '45+'])

In [19]:
# concatemos de nuevo los tres dataframes y obtenemos el dataframe final con las anomalías resueltas
datos_final_df = pd.concat([datos_final1_df, datos_final2_df, datos_final3_df], axis=1)

In [20]:
datos_final_df.head()

,Puesto,Titulo,Año,Rating,Votos,Edad,Duración,Género,<18,18-29,30-44,45+
1,1,Cadena perpetua,1994,9.3,"2,201,192",13,2h 22min,Drama,9.4,9.3,9.3,9.1
2,2,El padrino,1972,9.2,"1,516,621",18,2h 55min,"Crime, Drama",9.2,9.2,9.2,9.1
3,3,El padrino: Parte II,1974,9.0,"1,062,493",18,3h 22min,"Crime, Drama",9.1,9.1,9.0,8.9
4,4,El caballero oscuro,2008,9.0,"2,182,674",12,2h 32min,"Action, Crime, Drama",9.3,9.2,9.0,8.4
5,5,12 hombres sin piedad,1957,8.9,"642,713",A,1h 36min,Drama,9.3,9.1,8.9,8.7


In [21]:
# Ahora ya podemos realizar alguna búsqueda, como filtrar por el año de la película, por ejemplo
pelis_2015 = datos_final_df[datos_final_df['Año'] == '2015']
pelis_2015

,Puesto,Titulo,Año,Rating,Votos,Edad,Duración,Género,<18,18-29,30-44,45+
156,156,Del revés (Inside Out),2015,8.2,"570,569",A,1h 35min,"Animation, Adventure, Comedy",7.9,8.3,8.1,7.9
163,163,La habitación,2015,8.1,"339,810",12,1h 58min,"Drama, Thriller",8.3,8.3,8.0,7.9
202,202,Mad Max: Furia en la carretera,2015,8.1,"824,370",16,2h,"Action, Adventure, Sci-Fi",8.4,8.3,8.0,7.6
224,224,Spotlight,2015,8.1,"389,574",12,2h 9min,"Biography, Crime, Drama",7.9,8.2,8.0,8.0
249,249,Drishyam,2015,8.2,"60,285",NC,2h 43min,"Crime, Drama, Mystery",8.2,8.5,7.9,6.4


In [25]:
# Podemos comprobar como los datos que antes estaban erróneos, ahora ya estan corregidos
edad_NC = datos_final_df[datos_final_df['Edad'] == 'NC']
edad_NC

,Puesto,Titulo,Año,Rating,Votos,Edad,Duración,Género,<18,18-29,30-44,45+
33,33,Harakiri,1962,8.7,"31,150",NC,2h 13min,"Action, Drama, History",7.3,8.8,8.7,8.5
93,93,Dangal,2016,8.4,"136,974",NC,2h 41min,"Action, Biography, Drama",8.6,8.6,8.3,7.4
132,132,Masacre (ven y mira),1985,8.3,"48,480",NC,2h 22min,"Drama, War",8.4,8.5,8.2,8.1
135,135,Ran,1985,8.2,"104,312",NC,2h 42min,"Action, Drama",8.2,8.3,8.2,8.2
146,146,El tesoro de Sierra Madre,1948,8.2,"107,984",NC,2h 6min,"Adventure, Drama, Western",8.5,8.2,8.1,8.3
152,152,Mi padre y mi hijo,2005,8.3,"72,616",NC,1h 48min,"Drama, Family",8.0,8.5,8.3,7.3
171,171,Fresas salvajes,1957,8.2,"88,704",NC,1h 31min,"Drama, Romance",8.2,8.3,8.1,8.1
172,172,La ley del silencio,1954,8.1,"134,128",NC,1h 48min,"Crime, Drama, Thriller",8.1,8.2,8.1,8.2
176,176,Stalker,1979,8.2,"105,434",NC,2h 42min,"Drama, Sci-Fi",8.7,8.4,8.1,7.8
182,182,Eskiya,1996,8.3,"58,945",NC,2h 8min,"Crime, Drama, Thriller",6.9,8.4,8.4,6.9


In [23]:
datos_final_df

,Puesto,Titulo,Año,Rating,Votos,Edad,Duración,Género,<18,18-29,30-44,45+
1,1,Cadena perpetua,1994,9.3,"2,201,192",13,2h 22min,Drama,9.4,9.3,9.3,9.1
2,2,El padrino,1972,9.2,"1,516,621",18,2h 55min,"Crime, Drama",9.2,9.2,9.2,9.1
3,3,El padrino: Parte II,1974,9.0,"1,062,493",18,3h 22min,"Crime, Drama",9.1,9.1,9.0,8.9
4,4,El caballero oscuro,2008,9.0,"2,182,674",12,2h 32min,"Action, Crime, Drama",9.3,9.2,9.0,8.4
5,5,12 hombres sin piedad,1957,8.9,"642,713",A,1h 36min,Drama,9.3,9.1,8.9,8.7
...,...,...,...,...,...,...,...,...,...,...,...,...
246,246,Contratiempo,2016,8.1,"104,343",12,1h 46min,"Crime, Mystery, Thriller",8.2,8.3,8.0,7.8
247,247,Guardianes de la galaxia,2014,8.0,"1,000,430",12,2h 1min,"Action, Adventure, Comedy",8.3,8.1,8.0,7.9
248,248,Neon Genesis Evangelion: The End of Evangelion,1997,8.1,"33,330",13,1h 27min,"Animation, Action, Drama",8.5,8.4,8.0,6.7
249,249,Drishyam,2015,8.2,"60,285",NC,2h 43min,"Crime, Drama, Mystery",8.2,8.5,7.9,6.4


In [24]:
# guardamos en *.csv los datos extraidos ya corregidos
datos_final_df.to_csv('top_peliculas.csv', sep = ';', index=False, header=False)